Para conectarnos a una base de datos local, vamos a usar pandas junto con SQLAlchemy

In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
from sqlalchemy.orm import sessionmaker

definimos una funcion que nos permite armar una sesion de conexion con nuestra base de datos. Necesitamos para esto los datos para conectarnos. 

In [10]:
def get_session():
    dialect = 'postgresql' # este parametro puede cambiar dependiendo del motor de base de datos que se este utilizando
    user = 'admin'
    password = 'admin'
    host = 'localhost'
    port = '5432'
    database = 'default'
    url = f"{dialect}://{user}:{password}@{host}:{port}/{database}" # ?sslmode=require

    # Establishing a connection to the database
    engine = create_engine(url)
    Session = sessionmaker(bind=engine)
    session = Session()

    return session, engine

definimos otra funcion que lo unico que hara sera cerrar la sesion abierta

In [3]:
def close_session(session):
    session.close()

Finalmente creamos una funcion mas para correr queries en nuestra db independizandonos de los metodos y parametros particulares de SQLAlchemy

In [18]:
def run_query(query, engine):
    with engine.connect() as connection:
        result = connection.execute(query)
    return result

Vamos a construir algunos queries y consultar a nuestra base de datos para explorarla. Si acabamos de crear nuestra base de datos, deberia estar vacia, sin tablas de usuario y con un unico usuario 'admin'

In [26]:
# vamos a crear la sesion una vez para evitar crearla cada vez que se necesite hacer una consulta
session, engine = get_session()

In [27]:
query = 'SELECT usename, usesuper, usecreatedb FROM pg_user;'
result = run_query(query, engine)

# vamos a convertir el resultado de la consulta en un DataFrame
df = pd.DataFrame(result.fetchall(), columns=result.keys())
print(df)


  usename  usesuper  usecreatedb
0   admin      True         True


para evitar repetir mucho codigo, vamos a encapsular la accion anterior y construir una funcion que retorne un dataframe a partir de un query y el engine iniciado

In [28]:
def df_from_query(query):
    result = run_query(query, engine)
    return pd.DataFrame(result.fetchall(), columns=result.keys())

Probemos algunos queries mas para explorar la base de datos creada

In [29]:
# cantidad de esquemas en nuestra bd
df_from_query('SELECT COUNT(*) AS total_schemas FROM information_schema.schemata;')


,total_schemas
0,4


In [30]:
# cuales son los esquemas que tenemos?
df_from_query('SELECT schema_name \
FROM information_schema.schemata \
ORDER BY schema_name;')

,schema_name
0,information_schema
1,pg_catalog
2,pg_toast
3,public


In [32]:
# listar tablas de sistema y de usuario.
df = df_from_query("SELECT  \
                    schemaname, \
                    tablename, \
                    tableowner, \
                    CASE  \
                        WHEN schemaname IN ('pg_catalog', 'information_schema') THEN 'Sistema' \
                        ELSE 'Usuario' \
                    END AS tipo \
                FROM pg_tables \
                ORDER BY tipo, schemaname, tablename;")
df

            schemaname                tablename tableowner     tipo
0   information_schema             sql_features      admin  Sistema
1   information_schema  sql_implementation_info      admin  Sistema
2   information_schema                sql_parts      admin  Sistema
3   information_schema               sql_sizing      admin  Sistema
4           pg_catalog             pg_aggregate      admin  Sistema
..                 ...                      ...        ...      ...
63          pg_catalog               pg_ts_dict      admin  Sistema
64          pg_catalog             pg_ts_parser      admin  Sistema
65          pg_catalog           pg_ts_template      admin  Sistema
66          pg_catalog                  pg_type      admin  Sistema
67          pg_catalog          pg_user_mapping      admin  Sistema

[68 rows x 4 columns]


In [33]:
# podemos con pandas saber cuantas tablas son de sistema y cuantas de usuario
df['tipo'].value_counts()

tipo
Sistema    68
Name: count, dtype: int64

Si todavia no creamos ninguna tabla, es normal que solo existan tablas de sistema

Veamos las 10 tablas mas pesadas

In [34]:
df_from_query("SELECT  \
                    schemaname || '.' || tablename AS tabla_completa, \
                    pg_size_pretty(pg_total_relation_size(schemaname || '.' || tablename)) AS tamaño \
                FROM pg_tables \
                ORDER BY pg_total_relation_size(schemaname || '.' || tablename) DESC \
                LIMIT 10;")

,tabla_completa,tamaño
0,pg_catalog.pg_proc,1216 kB
1,pg_catalog.pg_rewrite,728 kB
2,pg_catalog.pg_attribute,704 kB
3,pg_catalog.pg_description,616 kB
4,pg_catalog.pg_statistic,288 kB
5,pg_catalog.pg_depend,272 kB
6,pg_catalog.pg_collation,240 kB
7,pg_catalog.pg_operator,232 kB
8,pg_catalog.pg_type,232 kB
9,pg_catalog.pg_class,232 kB


Este query puede llegar a ser util para monitoreo

In [ ]:
close_session(session) # cerramos la sesion

## Ejercicio propuesto

Cargar en la base de datos el dataset de brazil-weather. Cargar las tres tablas, la de informacion de estaciones o un sample, si no tienen suficiente espacio, los codigos de las estaciones y los codigos de viento. 

ayuda: Hay mas de una manera de hacer esto. Pueden convertir el csv e insertarlo con SQL, o bien pueden usar pandas y SQLAlchemy para transformarlo desde un dataframe. 